In [0]:
import torch
print(torch.__version__)
import tensorflow as tf

1.0.1.post2


In [0]:
# torch.tensor(data) creates a torch.Tensor object with the given data.
V_data = [1., 2., 3.]
V = torch.tensor(V_data)
print(V)

# Creates a matrix
M_data = [[1., 2., 3.], [4., 5., 6]]
M = torch.tensor(M_data)
print(M)

# Create a 3D tensor of size 2x2x2.
T_data = [[[1., 2.], [3., 4.]],
          [[5., 6.], [7., 8.]]]
T = torch.tensor(T_data)
print(T)

tensor([1., 2., 3.])
tensor([[1., 2., 3.],
        [4., 5., 6.]])
tensor([[[1., 2.],
         [3., 4.]],

        [[5., 6.],
         [7., 8.]]])


In [0]:
# Index into V and get a scalar (0 dimensional tensor)
print(V[0])
# Get a Python number from it
print(V[0].item())

# Index into M and get a vector
print(M[0])

# Index into T and get a matrix
print(T[0])

tensor(1.)
1.0
tensor([1., 2., 3.])
tensor([[1., 2.],
        [3., 4.]])


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## size testing


###torch

In [0]:
p = torch.tensor([
    [[1,1],[1,1],[1,1],[1,1],[1,1]],
    [[1,1],[1,1],[1,1],[1,1],[1,1]],
    [[1,1],[1,1],[1,1],[1,1],[1,1]]
])
p.size()

torch.Size([3, 5, 2])

In [0]:
list(p.size())

[3, 5, 2]

In [0]:
p_shape = list(map(int, p.size()))

In [0]:
axis = len(p_shape) - 1
axis

2

###tensorflow

In [0]:
# t = tf.constant([
#     [1.0, 2.0, 3.0], 
#     [4.0, 5.0, 6.0], 
#     [1,2,3], 
#     [1,2,3]
# ])
# t.get_shape()

t = tf.constant([
    [[1,1],[1,1],[1,1],[1,1],[1,1]],
    [[1,1],[1,1],[1,1],[1,1],[1,1]],
    [[1,1],[1,1],[1,1],[1,1],[1,1]],
    [[1,1],[1,1],[1,1],[1,1],[1,1]]
])
t.get_shape()

TensorShape([Dimension(4), Dimension(5), Dimension(2)])

In [0]:
t.get_shape().as_list()

[4, 5, 2]

In [0]:
t_shape = list(map(int, t.get_shape()))
t_shape

[4, 5, 2]

In [0]:
axis = len(t_shape) - 1

## reduce_max

### torch

In [0]:
x = torch.tensor([
    [1.0, 2.0, 3.0], 
    [4.0, 5.0, 6.0], 
    [7.0, 10.0, 2.0]
])

axis = len(x.size()) - 1
print(axis)
m = torch.max(x, dim=axis)[0]
m2 = torch.max(x, axis, keepdim=True)[0]
res = m + torch.log(torch.sum(torch.exp(x-m2), axis))
print(m)
print(m2)
print(res)

1
tensor([ 3.,  6., 10.])
tensor([[ 3.],
        [ 6.],
        [10.]])
tensor([ 3.4076,  6.4076, 10.0489])


### tensorflow

In [0]:
x = tf.constant([
    [1.0, 2.0, 3.0], 
    [4.0, 5.0, 6.0], 
    [7.0, 10.0, 2.0]
])

axis = len(x.get_shape()) - 1
print(axis)
m = tf.reduce_max(x, axis)
m2 = tf.reduce_max(x, axis, keep_dims=True)
res = m + tf.log(tf.reduce_sum(tf.exp(x-m2), axis))

with tf.Session() as sess:
  printm = sess.run(m)
  printm2 = sess.run(m2)
  printres = sess.run(res)
print(printm)
print(printm2)
print(printres)

1
[ 3.  6. 10.]
[[ 3.]
 [ 6.]
 [10.]]
[ 3.407606  6.407606 10.048907]


# fast-pixel-cnn (PyTorch 1.0)

#### Vivek Subramaniam (Digital Video and Multimedia Laboratory)
#### Mentor: Zheng Shou

This is a PyTorch 1.0 implementation of [Fast PixelCNN++](https://github.com/PrajitR/fast-pixel-cnn) by [@PrajitR](https://github.com/PrajitR/) found in ["Fast Generation for Convolutional Autoregressive Models" (2017)](https://arxiv.org/abs/1704.06001). It has been ported from TensorFlow 1.0 to PyTorch 1.0.